In [1]:
from huggingface_hub import login
login()

In [2]:
from  datasets  import  load_dataset

In [3]:
ds = load_dataset("bigcode/the-stack", data_dir="data/python", streaming=True, split="train")

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

In [4]:
true_ex = 'def str(val):\n    """Convert float to string, taking the locale into account."""\n    return _format("%.12g", val)\n\ndef colnum2name(n):\n    "Translate a column number to name (e.g. 1->\'A\', etc.)."\n    assert n > 0\n    s = ""\n    while n:\n        n, m = divmod(n-1, 26)\n        s = chr(m+ord(\'A\')) + s\n    return s\ndef cellname(x, y):\n    "Translate a cell coordinate to a fancy cell name (e.g. (1, 1)->\'A1\')."\n    assert x > 0 # Column 0 has an empty name, so can\'t use that\n    return colnum2name(x) + str(y)'
print(true_ex)

def str(val):
    """Convert float to string, taking the locale into account."""
    return _format("%.12g", val)

def colnum2name(n):
    "Translate a column number to name (e.g. 1->'A', etc.)."
    assert n > 0
    s = ""
    while n:
        n, m = divmod(n-1, 26)
        s = chr(m+ord('A')) + s
    return s
def cellname(x, y):
    "Translate a cell coordinate to a fancy cell name (e.g. (1, 1)->'A1')."
    assert x > 0 # Column 0 has an empty name, so can't use that
    return colnum2name(x) + str(y)


In [5]:
import re
def is_usefull_class(code):
  pattern = r'def\s+([^\s]+)\('
  func_names = re.findall(pattern, code)
  functions = code.split('def ')
  #if contains less than 2 functions - throw them 
  if len(re.findall(pattern, code)) < 2:
    return False
  #if first function is not used in next functions - throw them
  elif not any([func_names[0]+'(' in i for i in functions[2:]]):
    return False
  else: return True

def is_useful(code):
  # print(functions)
  #if more than 1 star - throw
  if code['max_stars_count'] and code['max_stars_count']> 1:
    return False

  classes= code["content"].split('class ')
  return any([is_usefull_class(cl) for cl in classes])

In [44]:
is_useful({"content": true_ex, 'max_stars_count':None})

True

In [7]:
import pandas as pd
df = pd.DataFrame(columns = ['code'])

In [45]:
for n, sample in enumerate(iter(ds)):
  if n > 100:
    break
  for e, cl in enumerate(sample["content"].split('class ')):
      if is_usefull_class(cl):
        df.loc[df.shape[0]] = ['class ' + cl if e > 0 else cl]

In [46]:
pattern = r'def\s+([^\s]+)\('
df['func_names'] = df['code'].apply(lambda x: re.findall(pattern, x))

In [47]:
df.head()

,code,func_names
0,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,"[calc_hash, compare, limit, resize, set_standard]"
1,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,"[read_val, read_row, read_grid, make_blank_row..."
2,import numpy as np\n\n\ndef train_ml_squarer()...,"[train_ml_squarer, square]"
3,class and set up the window\n super()._...,"[setup, on_draw, on_key_press, on_key_release,..."
4,class = serializers.GameSessionSerializer\n ...,"[get_parent_game, get_queryset, dispatch]"


Надо получить примеры как из базового датасета

In [48]:
import ast
import re

def extract_function_code_with_regex(code):
    pattern = re.compile(r'\bdef\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\((.*?)\)\s*:\s*(.*?)(?=def|$)', re.DOTALL)
    matches = pattern.findall(code)
    functions = {}

    for match in matches:
        function_name = match[0]
        function_args = match[1]
        function_body = match[2]
        functions[function_name] = f"def {function_name}({function_args}):\n    {function_body}"

    return functions

In [49]:
from transformers import LlamaForCausalLM, CodeLlamaTokenizer, StoppingCriteria, StoppingCriteriaList
import pandas as pd
MODEL_NAME = "codellama/CodeLlama-7b-hf"
tokenizer = CodeLlamaTokenizer.from_pretrained(MODEL_NAME)
# model = LlamaForCausalLM.from_pretrained(MODEL_NAME, load_in_8bit=True)
# model.config.forced_eos_token_id = [13]


In [1]:
def get_prompt(line):
  code = line['code']
  # func_names = line['func_names']
  target_functioons = []
  functions = extract_function_code_with_regex(code)
  for k,v in functions.items():
    splited = code.split(v)
    if len(tokenizer(code, return_tensors="pt")["input_ids"]) > 500:
        continue
    if k+'(' in splited[-1]:
      target_functioons.append(k)
    data.loc[data.shape[0]] = [splited[0]+v,k, splited[-1], code]
  # print(target_functioons)
  return True


In [14]:
data = pd.DataFrame(columns = ['prompt', 'target_function', 'code', 'full'])
data

,prompt,target_function,code,full


In [15]:
get_prompt(df.loc[0])

True

In [16]:
ex = data.loc[1]
print(ex['prompt'])

# UCF Senior Design 2017-18
# Group 38

from PIL import Image
import cv2
import imagehash
import math
import numpy as np

DIFF_THRES = 20
LIMIT = 2
RESIZE = 1000


def calc_hash(img):
    """
    Calculate the wavelet hash of the image
        img: (ndarray) image file
    """
    # resize image if height > 1000
    img = resize(img)
    return imagehash.whash(Image.fromarray(img))


def compare(hash1, hash2):
    """
    Calculate the difference between two images
        hash1: (array) first wavelet hash
        hash2: (array) second wavelet hash
    """
    return hash1 - hash2





In [17]:
print(f"We are expecting '{ex['target_function']}'")
print(ex['code'].split(ex['target_function']+'(')[0])

We are expecting 'compare'
def limit(img, std_hash, count):
    """
    Determine whether image should be removed from image dictionary in main.py
        img: (ndarray) image file
        std_hash: (array) wavelet hash of comparison standard
        count: (int) global count of images similar to comparison standard
    """
    # calculate hash for given image
    cmp_hash = calc_hash(img)

    # compare to standard
    diff = 


Ок, работает, запускаем по существующему

In [18]:
data = pd.DataFrame(columns = ['prompt', 'target_function', 'code', 'full'])
data

,prompt,target_function,code,full


In [63]:
import time
s  = time.time()
df.apply(get_prompt, axis = 1)
print(time.time() - s)

319.2361972332001


In [64]:
data

,prompt,target_function,code,full
0,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,calc_hash,"def compare(hash1, hash2):\n """"""\n Calcu...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...
1,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,compare,"def limit(img, std_hash, count):\n """"""\n ...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...
2,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,limit,"def resize(img):\n """"""\n Resize an image...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...
3,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,resize,"def set_standard(images, filename):\n """"""\n...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...
4,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,read_val,"def read_row():\n return list(map(int, inpu...",#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...
...,...,...,...,...
11276,#coding:utf-8\n# Copyright (c) 2019 PaddlePad...,hub_stat,\n,#coding:utf-8\n# Copyright (c) 2019 PaddlePad...
11277,import distro\nimport logging\nimport os\nimpo...,generate_meta_iso,def gentemp(prefix):\n return tempfile....,import distro\nimport logging\nimport os\nimpo...
11278,import distro\nimport logging\nimport os\nimpo...,upload_volume,"def handler(stream, nbytes, _):\n data ...",import distro\nimport logging\nimport os\nimpo...
11279,class MeanSquaredError(Loss):\n\n def call(...,__init__,defaults to `SUM_OVER_BATCH_SIZE`. When used w...,class MeanSquaredError(Loss):\n\n def call(...


In [79]:
data.shape

(8501, 6)

In [69]:
data['count'] = data.apply(lambda x:x['code'].count(x['target_function']) , axis = 1)

In [75]:
data = data[data['count'] > 0]
data = data.reset_index()

In [76]:
data

,index,prompt,target_function,code,full,count
0,0,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,calc_hash,"def compare(hash1, hash2):\n """"""\n Calcu...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...,2
1,1,# UCF Senior Design 2017-18\n# Group 38\n\nfro...,compare,"def limit(img, std_hash, count):\n """"""\n ...",# UCF Senior Design 2017-18\n# Group 38\n\nfro...,2
2,4,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,read_val,"def read_row():\n return list(map(int, inpu...",#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,2
3,5,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,read_row,def read_grid():\n return [read_row() for _...,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,1
4,6,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,read_grid,def make_blank_row(i):\n return [UNKNOWN] *...,#!/usr/bin/env python3\n\nUNKNOWN = -1\n\ndef ...,1
...,...,...,...,...,...,...
8496,11271,class SourcesTest(unittest.TestCase):\n def _...,_create_temp_file,def test_read_from_source(self):\n file_nam...,class SourcesTest(unittest.TestCase):\n def _...,3
8497,11275,#coding:utf-8\n# Copyright (c) 2019 PaddlePad...,get_stat_server,default_stat_config\n return server_list[ra...,#coding:utf-8\n# Copyright (c) 2019 PaddlePad...,1
8498,11277,import distro\nimport logging\nimport os\nimpo...,generate_meta_iso,def gentemp(prefix):\n return tempfile....,import distro\nimport logging\nimport os\nimpo...,1
8499,11278,import distro\nimport logging\nimport os\nimpo...,upload_volume,"def handler(stream, nbytes, _):\n data ...",import distro\nimport logging\nimport os\nimpo...,1


In [15]:
data['ids'] = data['full'].apply(lambda x: tokenizer(x, return_tensors="pt")["input_ids"].shape[-1])

In [22]:
data = data[data['ids'] < 500]

In [25]:
small_data = data[:1000]

In [ ]:
#small_data.to_csv('/home/sasha/effective-inference/clean_naming/small_data.csv')

In [37]:
def get_defenitions(line):
    code = line['code']
    # func_names = line['func_names']
    target_functioons = []
    functions = extract_function_code_with_regex(code)
    for k,v in functions.items():
        func_data.loc[func_data.shape[0]] = [v, k]
    # print(target_functioons)

In [38]:
func_data = pd.DataFrame(columns = ['func_definition', 'func_name'])

In [39]:
small_data.apply(get_defenitions, axis=1)


2       None
3       None
4       None
5       None
6       None
        ... 
2501    None
2515    None
2516    None
2525    None
2526    None
Length: 1000, dtype: object

In [35]:
# func_data.drop_duplicates().reset_index().to_csv('/home/sasha/effective-inference/clean_naming/big_func_data.csv')

In [31]:
# func_data.drop_duplicates().reset_index().to_csv('/home/sasha/effective-inference/clean_naming/small_func_data.csv')

In [40]:
func_data.drop_duplicates().reset_index()

,index,func_definition,func_name
0,0,"def read_row():\n return list(map(int, inpu...",read_row
1,1,def read_grid():\n return [read_row() for _...,read_grid
2,2,def make_blank_row(i):\n return [UNKNOWN] *...,make_blank_row
3,3,def make_blank_grid(n):\n return [make_blan...,make_blank_grid
4,4,def compute_max_path_sum(grid):\n memo = ma...,compute_max_path_sum
...,...,...,...
1107,3186,def dp(n):\n # 查备忘录，避免重复计算\n if ...,dp
1108,3189,def get_datetime():\n daytime = datetime.da...,get_datetime
1109,3190,def update_json(file=r'update.json'):\n inf...,update_json
1110,3192,def getPrinters():\n global printers\n\tif ...,getPrinters


In [22]:
print(func_data.loc[0]['func_definition'])

def compare(hash1, hash2):
    """
    Calculate the difference between two images
        hash1: (array) first wavelet hash
        hash2: (array) second wavelet hash
    """
    return hash1 - hash2



